# DNN训练流程

## I. 理解input data
<font color=blue>通过理解数据来得到一些“如果这个任务是人来完成的话会怎么做”的直觉认知。</font>
### step1：直接观察原始数据
 - 直接浏览数据（比如图片），尝试理解数据的分布，观察数据pattern。
   - 比如：duplicated examples，corrupted images/labels，找到data imbalances/bias
 - 可以自己用自己的方法先对数据做分类，这种个人直觉可能对后面选择model的结构提供思路。
   - 比如视觉任务中：
     - 自己先判断是不是local feature就足够了，还是需要global context
     - 有没有一些varince是直接用preprocess就能处理掉而不需要model来learn的
     - detail有多重要，downsample能做到什么程度

## II. 搭training + evaluation pipeline
 - 搭一个完整但简单的训练和评估框架，确认计算过程没有错误。
   - 这个阶段要用一个非常小的model，比如linear classfier或者tiny CNN。这可以减少构建pipeline过程中引入太多错误的机会。
   - 用小模型和少量样本做训练，visualize the loss, accuracy, model predictions来看模型的工作情况。
   - 在整个模型试训练的过程中，建模者可以根据自己对各个环节的假设，做一系列的ablation试验。

**这部分操作的要点包括：**
  1. fix random seed
  2. 关闭模型中所有fancy的设置。比如data augmentation。因为这是一种regularization策略，这个阶段放进来没有必要，只会增加产生bug的几率
  3. 这个阶段做evaluation要用整个test set，不要用少数batch
  4. 确认初始阶段的loss计算正确，sanity check的第一项
  5. 给weights好的init，比如回归的结果均值是50，那么最好把bias设为50，这样weights可以converge得更快。
  6. 找human baseline。找至少一个loss之外的,人可以直观理解的metric(比如accuracy),这样就可以用人的水平给这个metric定一个baseline。
  7. train一个input-independent baseline。典型方式是让所有的input value都为0，看loss是多少。然后再用真实的data作为input，看loss是不是比前面的loss低。这一步是为了确认model learn to extract information from input.
  8. 在一个batch上overfit
  9. 增加一点模型的capacity，确认training loss有下降。确认model在正确工作。
  10. 在将data放入model之前(before y_pred = model(x))对preprocess之后的data做visualize。
  11. 在训练过程中，用一组固定的test batch来visualize模型的预测结果。每次weights迭代之后，他们的预测结果就会发生变化，dynamics of how these predictions move会提供很好的关于训练进展情况的直觉。如果预测结果wiggles too much，说明模型不太稳定。learning rates太小或者太大也可以通过这个图看出来。
  12. 用backprop方法来确认网络中的dependency关系正确。网络中可能有大量的复杂矩阵操作，包括broadcast等。有的地方涉及矩阵的变换，如果需要copy某个矩阵的变形形态，但实际上用的是View就会出错，这些问题在写代码的时候很容易忽略。而且这类错误不会有语法报错，模型还是会继续train。
      - solution是临时将loss设置为简单的函数，比如最后一层score中第i行元素之和，然后执行backprop。这时候如果得到的所有dw中除了第i行之外其他行全都是0，那就说明矩阵之间的前后dependency关系是正确的。
      - 同样的方法还可以用来检验autoregressive model在time t的时候只依赖此前的time 1,2,3...,t-1时刻的取值。
      - 总的来说，gradients提供了网络结构中的dependency的信息

### step2：sanity check: 检查loss和gradient的计算是否正确
#### gradient check操作方式
1. 计算公式：
   1. 用centered形式的公式：$\frac{df(x)}{dx}=\frac{f(x+h)-f(x-h)}{2h}$
   2. 在比较解析解和数值解的时候，用relative error，不用absolute error。可以用：$$
      \color{Blue}{relative err = \frac{|f'_a-f'_n|}{max(eps, |f'_a|+|f'_n|)}, eps可以取1e-8}
      $$<font color=orange>分母中的eps用于防止两个值都为0的情况</font>
   3. 用<font color=blue>**双精度**</font>计算数值解
2. 做gradient check的时候要处理non-derministic effect in the network。
   - 会引入随机性的因素包括：dropout、random data augmentation。
   - 一种简单的处理方式是：关闭所有non-derministic effect in the network。但这种方式的坏处是这些操作本身是会影响梯度的，关闭了就不能确定加上这些操作时的梯度计算是否正确。
   - 因此，一种更好的方法是不关闭这些操作，而是<font color=blue>**fix random seed**来做gradient check</font>。
3. 至少做两次gradient check，第三次看网络复杂度。
   1. 先关闭regularization part检验loss part
      - 如果reg part太大的话，gradient得到的值中大部分来自reg part，那么真正的loss part可能已经错了，但是因为numerical issue而发现不了。
   2. 然后直接打开reg做check，或者单独做reg part的gradient check。
   3. <font color=orange>*(选做)让网络跑一小段时间，loss function已经开始下降之后再做gradient check。*</font>
      - 一开始就做gradient check的话，weight的初始值可能不好，导致check的结果是错的。
      - 比如：svm中如果一开始weight的初始值太小，那么得到的score矩阵上所有元素值都接近0，此时$max(0, \Delta-(s_{yi}-s_j))\approx \Delta$，所有dw都是0。很可能gradient原本是错误的，但是恰好也错成全0了，那么gradient check不会报错。
      - 如果试跑一段时间loss function开始下降之后再做gradient check就不会因为weight init的问题导致gradient check不到错误的gradient。
4. 当参数规模太大，做check的计算量太大时，不要check一部分参数的gradients，而是要check所有参数的部分dimension。
5. 结果判断
   1. 从结果来看，1e-7 通常是比较好的结果。1e-4 > relative error 在有kinks的目标函数中（e.g. ReLU）能接受。但是如果目标函数中没有用kink(e.g. tanh或softmax), 那么1e-4太高。
   2. 网络越深，relative error的合理值就越大。因为Forward计算中，每一层的矩阵乘法都有误差，越往后累积的误差越多。因此数值解的精度随着深度增加而增加。反方向上，梯度从最后一层往前传的时候每层也要做矩阵乘法，越往浅层累积的误差越多。所以，总的来说error会随着网络加深而累积。如果是10层以上的网络，1e-2 也可能是合理的。

#### 计算梯度时可能遇到的问题
1. 梯度值太小导致计算中出现numerical issue：<font color=green>最好print实际的gradient值看其取值范围</font>
   - 当某层矩阵的单个元素对应的gradient值非常接近0的时候，如果用的是minibatch，那么这些值还要$*\frac{1}{N}$就会更小。计算机受限于精度，当这些值非常小的时候，得到的计算结果可能有numerical issues，结果不准确。<font color=red>一般1e-10就可能有问题</font>
   - <font color=norange>处理方式：暂时把loss function用常数放大，用这种方式把梯度提高到1e0，也就是大约在1这个数量级上，再check gradient。</font>

2. Kinks(不可微的点)导致误报错
   - Kinks会造成gradient check出现两种梯度不一致报错，但实际的梯度计算没有问题。
     - 常见存在kinks的函数：ReLU，SVM，Max函数中都有这种点。注意relu是取0和元素值中值更高的点。max是取矩阵中特定axis上所有元素中值最大的那个点。在numpy中ReLU用的是maximum。
     - 例子：ReLU中，$x=-1e-6$时，analytic gradient是0，在numerical gradient计算时，可能得到非零结果，因为如果$h>1e-6$时，$f(x+h)>0$。
     - 在max(0,x)函数中判断是否出现了kinks的方法是，看$f(x+h)$和$f(x-h)$的符号是否相反。
   - <font color=green>**solution**：使用少量的（2~3个）datapoints来做gradient check。这时候出现kinks的概率会比较小。</font>
   - <font color=red>注：不要通过降低h的值来减少kinks，如果h值太小，那么$f(x+h)$和$f(x-h)$的计算本身就会有精度问题。</font>假如把h从1e-4降低到1e-6，可能马上gradient check中的报错就都消失了，这时候可能就是h的值太小了。

#### 用简单的loss值查看网络是否配置正确
1. 用较小的weights来测试，看不计入reg item的loss是不是和预计的一样。**如：**C种类型的softmax的initial loss就是logC。
2. 加上reg item，增加reg strength应该看到loss增加。
3. 在小样本上overfit data, 初步确认网络有足够的表达力，且learning process没有问题。
   - 1. 关闭Regularization，在5-10个mini-batch上拿到100%的accuracy。一定要关掉reg，不然loss不可能取到0。
   - 2. 尝试不同的网络结构、learning rate，weight initialization。
   - <font color=blue>注：即使小样本上可以拿到loss为0，也不能保证网络的implementation是正确的。如果在小样本上overfit，但是全样本之后没有泛化，那么就可能implementation是有问题的。</font>
       - 比如：代码有问题导致所有的feature值都是random的，与input无关，这时候在小样本的training set上仍然可以拿到100%的正确率。但这个模型没有泛化能力。

## III. 迭代model
总体上分两步来完成：
 - 先用一个大到已经overfit的模型。<font color=norange>**拿到足够好的training loss**</font>
 - 然后为它找到合适的regularization。<font color=norange>**舍弃一部分training loss来提升valication loss**</font>

**第一个环节选model的技巧和注意事项：**
1. 一开始先用现成的model，不要一开始就想自创模型。**don't be a hero！**
2. 先用adam做优化器，CNN中SGD+V版Momentum会比adam好一点，但调参更费事。
3. 如果要给模型增加复杂的设计，一次只加一样东西。确保加上去之后loss有更好的结果了，再加新的东西。
4. 参考别人paper中的model时，不要直接复制别人的learning rate decay schedule。最好在一开始关闭learning rate decay。到模型基本都弄好了之后再来调这个参数。
   - 你的数据特征和别人的不同，这就使得loss下降的方式不一样
   - schedule一般都是选择达到在某个特定epoch之后做decay。这个选择和data数量直接相关。你的data数量也和别人不一样。

**第二个环节做regularization<font color=green>(在下面"Babysitting learning过程 + 调参"的step4中做)</font>的技巧和注意事项：**
1. 拿到更多数据永远是最好的regularization策略。另一个比其他策略都好的方法是ensemble，但是可能成本很高。还有一个ensemble不如more data的原因是，一般加到5个model之后，边际收益就很少了。
2. data augmentation
3. creative augmentation。比如用GAN来生成数据，或者用一些simulation数据。
4. 用pretrain network
5. 不要轻易使用unsupervised model
6. 降低input的维度。当data set小的时候，多余的维度会增加overfit的机会。所以，如果对你的任务而言，low-level details不重要的话，那就尽量用小一点的image做input。
7. 用drop的时候要注意它有时候和batchnorm一起连用的效果不好
8. 要用weight decay。
9. 要用early stopping。
10. 尽量尝试更大的model。larger model最终会有更大的overfit，但大模型的early stopped performance很可能比小模型的要好。
11. image任务中，画第一层的weights看结果是否合理。如果第一层weights画出来很像noise，那么模型很可能出问题了。

### step3. Babysitting learning过程 + 调参
- **step1: find LR that makes loss go down，**找到能让loss在100个iteration内快速下降的lr范围
  - **方法：**用所有的training data，较小的weight decay，画loss curve，手动选几个不同的learning rate来尝试。
  - **默认尝试值：**1e-1, 1e-2, 1e-3, 1e-4
    
- **step2: coarse grid，在step1选中的learning rate附近选2、3个值，以及2、3个weight decay**
  - **方法：**用<font color=green>**random search**</font>训练1~5个epoch，找到结果中比较好的值.
    - <font color=blue>用random search而不是grid search。因为模型结果对不同的参数敏感度不同。如果参数A的变化对模型结果影响很大，另一个参数B对模型的影响不大，那么在给定的cost(测试的次数)中，应该尽量多试几个不同的参数A。</font>
  - **默认weight decay：**1e-4, 1e-5, 0。如果网络复杂度相对数据量较高，weight decay可以大一点，相反可以小一点
    
- **step3: refine grid**
  - **方法：**从上一步中选出最好的参数值，跑10~20个epoch。这一步不用learning rate decay
    
- **step4: 画图，check los curve和acc curve，选择lr decay schedule**
  - 根据learn curve的形态来调整模型的配置：
    - 1. 在loss还很大的时候就出现plateau，做learning rate decay
    - 2. training accuracy和validation accuracy太接近，underfit：用更大的model，训练更长的时间，也可以考虑增加learning rate。
    - 3. training accuracy和validation accuracy差距太大，overfit：加大weight decay或者其他regulation，增加数据。
  - 如果learning rate出现了突然的divergence，可能是模型有问题，比如梯度消失，training sample中有异常样本，比如损坏的数据等。
    
- **step5: 选好配置后完整跑一遍模型**
- **step6：如果已经拿到满意的结果就结束，否则GOTO step4**

#### <font color=blue>learning过程中要跟踪的图</font>
1. 画loss curve，看学习过程有没有在持续降低loss
   - 横轴要用epoch，不要用iteration。单次iteration的结果受batch的大小影响，epoch没有这个问题
   - loss curve上的loss值上下幅度太大的话，说明batch太小。
   - loss curve可以画log形态的，原因是learning过程中loss的收敛是指数级的。画成log的话得到的应该是偏直线的形态。
2. 画training acc和validation acc，用这两条线之间的gap来判断overfitting
   - gap太大说明overfitting
   - gap太小说明underfitting，应该增加模型复杂度
3. 画每一层activation和gradient的直方图，看有没有saturate或者vanish，以便确认weight initialization是不是有问题
4. 画first-layer visualization，看网络学到了什么东西
5. 看每个weights每一次迭代变化值$dw$的scale相对迭代前$w$的scale的比例。这个值一般在1e-3附件比较合理。如果低于这个值，那么learning rate可能太小了，反之则反。

## part3: squeeze out the juice
### step4. Model Ensemble
1. 同一个模型，不同的初始化值
   - 用Cross-validation来决定最好的超参数，然后多次reinitialization randomly得到多个结果
2. 同一个模型，选择Cross-validation中最好的前几个结果，每个对应一个模型，组成ensemble
3. [低成本]用同一个模型的不同checkpoint来组成ensemble。主要用在训练成本很高的时候
4. [低成本]在训练过程中，记录下exponentailly decaying sum of previous weights during training. 相当于average the state of network over last several iterations.通常这种方式可以获得一点准确率的提升

5. 如果ensemble多个模型导致test time的计算量太大，可以<font color=green>**distilling the ensemble into a network**</font>

参考资料：
1. A Recipe for Training Neural Networks,2019 http://karpathy.github.io/2019/04/25/recipe/
2. cs231n official notes，NN part3，2015